# The code generates Land-Water masks from Sentinel-1 RTC images

- Downloaded Sentinel-1 RTC backscatter images at 10m resolution [VV Polariation, dB scale, Lee filter applied].


In [ ]:
import numpy as np
from osgeo import gdal,ogr, gdalconst
import subprocess
import glob
import os
from datetime import datetime
import matplotlib.pyplot as plt
import utm
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
from skimage.filters import threshold_otsu

## Point to folder with RTC images

In [ ]:
folder_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_598_DPHR/VV_tifs'
slc_list = glob.glob(folder_dir+'/*_VV.tif');

for slc in slc_list:
    
    date = os.path.basename(slc).split('_')[2][0:8]
    print(date)

## Merge and subset SLCs

- Here we merge multiple SLC frames and subset to the region of interest (ROI).


In [ ]:
def extract_date_from_filename(filename):
    date_str = os.path.basename(filename).split('_')[2][0:8];
    try:
        return datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        print(f"Failed to parse date from filename: {filename}")
        return None
    
def find_nearest_date_file(directory_path, target_date):
    nearest_date_diff = None
    nearest_date_file = None

    for filename in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, filename)) and filename.lower().endswith('.tif'):
            extracted_date = extract_date_from_filename(filename)
            if extracted_date:
                date_diff = abs(extracted_date - target_date);#print(extracted_date,target_date,date_diff)
                if nearest_date_diff is None or date_diff < nearest_date_diff:
                    nearest_date_diff = date_diff
                    nearest_date_file = os.path.join(directory_path, filename)
    return nearest_date_file

####### Merged SLCs from different tracks #######

folder_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_598_DPHR/VV_tifs';
target_path = '/Users/bvarugu/Documents/Belomonte/Xingu_68_599_DPHR/VV_tifs';
merged_file_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs'
slc_list = glob.glob(folder_dir+'/*_VV.tif');

for slc in slc_list:
    slc1_date = extract_date_from_filename(slc);
    slc_file_2 =  find_nearest_date_file(target_path, slc1_date);
    slc2_date = extract_date_from_filename(slc_file_2);
    out_name = merged_file_dir+'/'+slc1_date.strftime("%Y%m%d")+'_604_599_merged_VV.tif'
    command = f"gdal_merge.py -o {out_name} {slc} {slc_file_2}"
    #print(command.split())

    subprocess.run(command.split())



#####subset SLCs from different tracks##########

merged_file_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs'
merged_slc_list = glob.glob(merged_file_dir+'/*_VV.tif');
subset_file_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs/subset_tifs';
bbox = [333550,9588140,441429,9800000]  # Example bounding box (min lon, min lat, max lon, max lat)
for slc in merged_slc_list:
    
    output_path = subset_file_dir+'/'+'subset_'+os.path.basename(slc);
    command = 'gdalwarp -te {} {} {} {} {} {}'.format(bbox[0],bbox[1],bbox[2],bbox[3],slc,output_path);
    subprocess.run(command.split());

print("Subsetting completed!")

## Get dynamic threshold to make land-water masks
- To know pixels that are consistenly water, we use the water frequency raster from Fassoni-Andrade, A. C., & Paiva, R. C. D. (2019). The map has frequency that a pixel has been identified as water in 15 years between 2003-2017.

- We consider pixels that are 50% frequency, i.e. pixels identified as water for half of the time
- The water frequency raster is resampled to match the SAR pixels

In [ ]:
# get water frequency file and resample it to subsetted RTC image
waterFreq_raster_file = '/Users/bvarugu/Documents/Belomonte/GIS_layers/open_water_frequency_Xingu_clip.tif'
sample_rtc_file = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs/subset_tifs/subset_202310_VV.tif'
with rasterio.open(sample_rtc_file) as src_rtc:
    rtc_data = src_rtc.read(1)  # Assuming single band data
    rtc_profile = src_rtc.profile
    rtc_crs = src_rtc.crs
    rtc_transform = src_rtc.transform
    rtc_width = src_rtc.width
    rtc_height = src_rtc.height
with rasterio.open(waterFreq_raster_file) as src_waterFreq:
    # Calculate the transform for the resampled data
    dst_transform, dst_width, dst_height = rasterio.warp.calculate_default_transform(
        src_waterFreq.crs, rtc_crs, src_waterFreq.width, src_waterFreq.height, *src_waterFreq.bounds, 
        dst_width=rtc_width, dst_height=rtc_height, resolution=None
    )

    # Create an empty array to store the resampled data
    waterFreq = np.empty((rtc_height, rtc_width), dtype=src_waterFreq.dtypes[0])

    # Perform the resampling
    rasterio.warp.reproject(
        source=rasterio.band(src_waterFreq, 1),
        destination=waterFreq,
        src_transform=src_waterFreq.transform,
        src_crs=src_waterFreq.crs,
        dst_transform=dst_transform,
        dst_crs=rtc_crs,
        resampling=Resampling.nearest  # Adjust as needed
    )

## Generate land water mask
- We fit a normal distribution to the backscatter of the pixels that are idenfied with >50% frequency in the water frequency raster. We compute the mean and standard deviation (std) from the fit. 
- We derive a threshold backscatter as: otsu_thrshold = mean + 2*std
- Pixels with backscatter less than the otsu_thrshold are identifed as water and the remaining is land

In [ ]:
def get_land_water_mask_otsu(rtc_image_path,waterFreq,max_freq,min_freq,save=None,output_mask_path=None):
    #read RTC image
    dataset = gdal.Open(rtc_image_path);
    if dataset is None:
        raise Exception("Failed to open image:", rtc_image_path)

    # Get image data and band information
    band = dataset.GetRasterBand(1)  # Assuming VH polarization is in the first band
    rtc_data = band.ReadAsArray();
    nodata_value = band.GetNoDataValue()
    
    #extract pixels to generate mask
    mask = (waterFreq >= max_freq) & (rtc_data>-50);
    
    # Apply the mask to extract backscatter values
    backscatter_values = rtc_data[mask];
    backscatter_values = backscatter_values[backscatter_values<0];

    mu1, std1 = norm.fit(backscatter_values.flatten());
    otsu_threshold = mu1+(2*std1);

    # Apply Otsu's threshold to create a binary mask
    binary_mask[rtc_data < otsu_threshold] = 1  # Apply threshold
    binary_mask = binary_mask.astype(int)
    binary_mask[rtc_data == -9999] = -9999
    

    if save==True:
        driver = gdal.GetDriverByName("GTiff");
        options = ["COMPRESS=LZW"];
        geotransform = dataset.GetGeoTransform();
        projection = dataset.GetProjection();
        dset = driver.Create(output_mask_path, binary_mask.shape[1], binary_mask.shape[0], 1, gdal.GDT_Byte, options);
        dset.SetGeoTransform(geotransform);
        dset.SetProjection(projection);
        band = dset.GetRasterBand(1)
        band.WriteArray(binary_mask)
        band.SetNoDataValue(-9999)  # Set no data value as -9999
        dset.FlushCache()  # Ensure data is written to disk
        # Close the dset
        dset = None
        
    # Close the dataset
    dataset = None
    
    return binary_mask, otsu_threshold 
subset_file_dir = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs/subset_tifs';
subset_slc_list = sorted(glob.glob(subset_file_dir+'/subset_*0_VV.tif'));

mask_dict = {}
for slc in subset_slc_list:
    output_mask = '/Users/bvarugu/Documents/Belomonte/Xingu_68_604_599_merged/VV_tifs/subset_tifs/'+'LW_mask_'+os.path.basename(slc)[6:]
    mask_name = os.path.basename(slc)[7:13];
    land_water_mask, otsu_threshold = get_land_water_mask_otsu(slc,waterFreq,50,1,save=True,output_mask_path=output_mask);
    
    mask_dict[mask_name] = {
        'mask': land_water_mask,
        'otsu_threshold': otsu_threshold,
        # Add any other relevant metadata
    }

## Make a Folium map for display
- optional from here

In [ ]:
def get_geo_bounds(raster):
    dataset = gdal.Open(raster);
    ulx, xres, xskew, uly, yskew, yres  = dataset.GetGeoTransform();
    #print(dataset.GetProjection())
    lrx = ulx + (dataset.RasterXSize * xres);
    lry = uly + (dataset.RasterYSize * yres);
    dataset=None
    
    uly, ulx = utm.to_latlon(ulx, uly, 22, northern=False);
    lry, lrx = utm.to_latlon(lrx, lry, 22, northern=False);
    return ulx, uly , lrx, lry

geo_box = get_geo_bounds(output_mask);
print(geo_box)
    
    

In [ ]:
## Make folium map
import folium
import branca
from folium import plugins
from matplotlib import cm
from PIL import ImageColor

lat_mean = np.mean([geo_box[1],geo_box[3]]).astype(np.float64)
lon_mean = np.mean([geo_box[0],geo_box[2]]).astype(np.float64)
map_bounds = [[geo_box[1], geo_box[0]],
             [geo_box[3], geo_box[2]]];

#Folium Map
map = folium.Map(width=1100, height=600, 
                 location=(lat_mean, lon_mean), 
                 zoom_start=10,
                 tiles='Stamen Terrain',attr='https://mt1.google.com/');
#Basemaps
#Add custom basemaps
for basemap in basemaps:
    basemaps[basemap].add_to(map)

palette = ['blue', 'green']#[::-1]  
cmap = branca.colormap.LinearColormap(colors=palette,
                         vmin=0,
                         vmax=1,
                         caption='Image Colormap')

folium.raster_layers.ImageOverlay(land_water_mask,
                                  opacity=0.5,
                                  bounds=map_bounds,
                                  name='2016_mask',
                                  colormap=cmap).add_to(map);
#Colorbar
folium_colormap = map.add_child(cmap)
# #layer Control
# map.add_child(folium.LayerControl())
#Display map
map

In [ ]:
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = False,
        control = True,
        show = False,
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        opacity = 0.8,
        show = False
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True,
        show = False,
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        opacity = 0.8,
        show = False
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True,
        opacity = 0.8,
        show = True
    )
}